# BO runs

In [23]:
import torch
from botorch.models import FixedNoiseGP, SingleTaskGP
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.acquisition.analytic import ExpectedImprovement
import numpy as np
import pickle
import sys
import time

load data from `prepare_Xy.ipynb`

In [24]:
X = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['X']
y = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['y']
nb_data = np.size(y)
nb_data

69839

convert to torch tensors

In [25]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

In [26]:
X.size()

torch.Size([69839, 12])

In [27]:
y.size()

torch.Size([69839, 1])

number of COFs for initialization

In [31]:
nb_COFs_initialization = 10

In [29]:
def bo_run(nb_iterations):
    # select initial COFs for training data randomly
    ids_acquired = np.random.choice(np.arange((nb_data)), size=nb_COFs_initialization, replace=False)

    # initialize acquired X, y
    X_acquired = X[ids_acquired, :]
    y_acquired = y[ids_acquired]
    # standardize outputs
    y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
    print(y_acquired.size())
    
    for i in range(nb_COFs_initialization, nb_iterations):
        print("iteration:", i)
        # construct and fit GP model
        model = SingleTaskGP(X_acquired, y_acquired)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_model(mll)

        # compute aquisition function at each COF in the database
        acquisition_function = ExpectedImprovement(model, best_f=y_acquired.max().item())
        acquisition_values = acquisition_function.forward(X.unsqueeze(1))

        # select COF with maximal aquisition value, which is not in the acquired set already
        # find COF with highest value of acquisition function, not in acquired set already
        ids_sorted_by_aquisition = acquisition_values.argsort(descending=True)
        for id_max_aquisition_all in ids_sorted_by_aquisition:
            if not id_max_aquisition_all.item() in ids_acquired:
                id_max_aquisition = id_max_aquisition_all.item()
                break

        # acquire this COF
        ids_acquired = np.concatenate((ids_acquired, np.array([id_max_aquisition])))

        # update X, y acquired
        X_acquired = torch.cat([X_acquired, X[id_max_aquisition, :].unsqueeze(0)])
        y_acquired = y[ids_acquired, :] # start over to normalize y properly
        y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
        print("")

        print("\tacquired COF", id_max_aquisition, "with y = ", y[id_max_aquisition])
        print("\tbest y acquired:", y[ids_acquired].max())
    return ids_acquired

In [36]:
nb_runs = 3
nb_iterations = 11
for r in range(nb_runs):
    ids_acquired = bo_run(nb_iterations)
    print(ids_acquired)
    torch.save({'ids_acquired': ids_acquired}, 'bo_run' + str(r) + '.pkl')

torch.Size([10, 1])
iteration: 10

	acquired COF 44897 with y =  tensor([183.6189], dtype=torch.float64)
	best y acquired: tensor(183.6189, dtype=torch.float64)
[34647 35678 69727 52355 23924 10911 42976 54307 59610 62690 44897]
torch.Size([10, 1])
iteration: 10

	acquired COF 5174 with y =  tensor([190.2927], dtype=torch.float64)
	best y acquired: tensor(190.2927, dtype=torch.float64)
[ 5125 54841 57876 20364 16192 11446 30070 25210 15672 44596  5174]
torch.Size([10, 1])
iteration: 10

	acquired COF 44897 with y =  tensor([183.6189], dtype=torch.float64)
	best y acquired: tensor(183.6189, dtype=torch.float64)
[ 4354 63357 16040 50514 55005 43266 11741 11211 47269 46420 44897]
